<a href="https://colab.research.google.com/github/asmayamani/DUA/blob/main/DUA_Active_Learning_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Active Learning Selection of training file

Active learning originaly aims to maximize the information gained from labeling at a limited annotation budget. In this applictaion, as we noticed the classes are highley imbalanced, and the presence of images and large objects all problems thata could hender the model's ability to generalize at deployment.

In this Notebook we present the Diversity in Uncertinity Aggregation (DUA) Active learning approach in combination with Weighting by Random Sample Performance (WRP).



1.   It first evaluate the model from the previous iteration on a random subset of the training data that hasn't been used for trianing yet.
2.   It then calculated the uncertinity of the inference made on the training data that hasn't been used for trianing yet, by summing the average uncertinities per class for each image
3.  After that it multiplies the weights with uncertinities and produce the final uncertinity score per image.
4. Finally, the images with the highest uncertinities are included in the new training cycle.
5. The model is build using the selected images using the notebook in [here](https://github.com/asmayamani/DUA/blob/main/Active_Learning_Training.ipynb)

It assumes an initial model and initial labled pool


#Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)


Mounted at /content/drive


In [ ]:
import os, glob, random, shutil
from pathlib import Path

In [ ]:
!git clone https://github.com/asmayamani/yolov7-AL
%cd yolov7-AL
!pip install -r requirements.txt

#Declerations

Important: Increase the itereation after completing each iteration

In [ ]:
#Enter the number iteration
iteration = 1

In [ ]:
main_dir = '/content/drive/MyDrive/AL/VOC2012/'
complete_data_folder = main_dir+'TrainF/'
main_exp_dir = 'DUA500/'
iteration_folder = 'Iteration_'+str(iteration)+'/'
complete_iteration_folder = main_dir+main_exp_dir+iteration_folder
os.makedirs(complete_iteration_folder) if not os.path.exists(complete_iteration_folder) else None
object_def = {'none':0,'aeroplane':1,'bicycle':2,'bird':3,'boat':4,'bottle':5,'bus':6,'car':7,'cat':8,'chair':9,'cow':10,'diningtable':11,
    'dog':12,'horse':13,'motorbike':14,'person':15,'pottedplant':16,'sheep':17,'sofa':18,'train':19,'tvmonitor':20}
approach = 'DUA'
weighting_approach = 'WRP'
number_of_random_images_from_training = 5000


In [ ]:
output_model = approach+str(iteration)+str(number_of_random_images_from_training)+str(weighting_approach)
previous_model_zip = 'Iteration_'+str(iteration-1)+'_Training_whole_best.zip'
budgetPerIteration = 500
expName = 'iteration'+str(iteration)+approach+'_'+str(number_of_random_images_from_training)
wexpName = 'w_iteration'+str(iteration)+approach+'_'+str(number_of_random_images_from_training)
previousExpfName = 'iteration'+str(iteration-1)+approach+weighting_approach+'_'+str(number_of_random_images_from_training)+'/'
previous_iteration_folder = 'Iteration_'+str(iteration-1)+'/'


#Definitions

The methods for active learning and some utility functions are defined here

In [ ]:
def DUA(fDir):
    classes = {}
    for i in range(len(object_def)):
      classes[str(i)] = {'objectsC' : 0, 'uncer' : 0}
    try:
      with open(fDir, 'r') as file2:  # read annotation.txt
          for row in [x.split(' ') for x in file2.read().strip().splitlines()]:
              classes[str(row[0])]['objectsC'] += 1
              classes[str(row[0])]['uncer'] += ((1-float(row[5])))
      file2.close()
      total = 0
      wAvg = 0
      for i in range(len(object_def)):
        # print(classes[str(i)])
        if classes[str(i)]['objectsC'] != 0:
          wAvg += ((classes[str(i)]['uncer']/classes[str(i)]['objectsC']) * classesWeight[str(i)])
          total += classes[str(i)]['objectsC']

      return(wAvg)
    except (IOError, OSError) as e:
        print(e)
        return(0)


In [ ]:
def getUncertinitiesForDir(dir, trainDir = [], valAccuracyDirF = '', method = 'avg', weighting = 1 , budget = 100):
  classesWeight = getWeightsForClassses(weighting, trainDir = trainDir, valAccuracyDirF = valAccuracyDirF)
  listofFilesNamesUncertinity = {}
  for f in glob.glob(dir+'/labels/*.txt'):
    namef = Path(f).name[:-4]
    fileFullDir = dir+'/labels/'+namef+'.txt'
    if method == 'DUA':
      listofFilesNamesUncertinity[namef] = DUA(fileFullDir,classesWeight)
  print(listofFilesNamesUncertinity)
  return sorted(listofFilesNamesUncertinity, key=listofFilesNamesUncertinity.get, reverse=True)[:budget]


In [ ]:
def getWeightsForClassses(weighting, trainDir = [], valAccuracyDirF = '', ):
  classesWeight = {}
  if weighting == 1:
    for i in range(len(object_def)):
      classesWeight[str(i)] = 1
  elif weighting == 3:
    classesWeight = applyMinMaxScalerToW(getWeightForValAccuracy(valAccuracyDirF))
  print(classesWeight)
  return classesWeight


In [ ]:
def newTraining(namefolder,setOfFileNames,oldTraning):
  added = 0
  path_label_2 = complete_iteration_folder+namefolder+'/labels/'
  path_images_2 = complete_iteration_folder+namefolder+'/images/'
  os.makedirs(path_label_2) if not os.path.exists(path_label_2) else None
  os.makedirs(path_images_2) if not os.path.exists(path_images_2) else None
  count = 0

  while (added < budgetPerIteration) and (count < len(setOfFileNames)):
    namef = setOfFileNames[count] #Path(setOfFileNames[count]).name[:-4]

    l_src_path = complete_data_folder+'labels/'+ namef+'.txt'
    i_src_path = complete_data_folder+'images/'+ namef+'.jpg'

    src_path = complete_data_folder+'labels/'+ namef+'.txt'
    if os.path.exists(l_src_path) and os.path.exists(i_src_path):
      dst_path = path_label_2 + namef+'.txt'
      shutil.copy(src_path, dst_path)
      src_path = complete_data_folder+'images/'+ namef+'.jpg'
      dst_path = path_images_2 + namef+'.jpg'
      shutil.copy(src_path, dst_path)
      added += 1
    count += 1

  for pth in oldTraning:
    # print(pth)
    namef = Path(pth).name[:-4]
    src_path = main_dir+main_exp_dir+previous_iteration_folder+previousExpfName+'labels/'+ namef+'.txt'
    dst_path = path_label_2 + namef+'.txt'
    shutil.copy(src_path, dst_path)

    src_path = main_dir+main_exp_dir+previous_iteration_folder+previousExpfName+'images/'+ namef+'.jpg'
    dst_path = path_images_2 + namef+'.jpg'
    shutil.copy(src_path, dst_path)


In [ ]:
def getWeightForValAccuracy(fDir, cnum = object_def):
    numberOfClasses = len(object_def)
    accuracyClasses = {}
    for i in range(numberOfClasses):
      accuracyClasses[str(i)] = 0
    try:
      with open(fDir, 'r') as file2:  # read annotation.txt
          first = True
          for row in [x.split(' ') for x in file2.read().strip().splitlines()]:
            if first:
              first = False
              continue
            if row == '':
              continue
            row = list(filter(lambda a: a != '', row))
            accuracyClasses[str(cnum[row[0]])] = float(row[5])


      file2.close()
      return(accuracyClasses)
    except (IOError, OSError) as e:
        print(e)


In [ ]:
def applyMinMaxScalerToW (wValCDic):
  minV = min(wValCDic[item] for item in wValCDic)
  maxV = max(wValCDic[item] for item in wValCDic)
  for i in range(len(object_def)):
    wValCDic[str(i)] = 1- ((wValCDic[str(i)] - minV)/(maxV - minV))
  return wValCDic

In [ ]:
def nominateImagesToObtainUncer(wasTrainedOnDir, experimentName, forWeights = False, fromRandSize = 500, seed = 42, trainingDir = complete_data_folder+'images/', weightedOnDir = '' ):
  #remove images that were trained on
  trainedOn = glob.glob(wasTrainedOnDir+'images/*.jpg')
  print(len(trainedOn))
  fullTrainSet = glob.glob(trainingDir+'*.jpg')
  orpNames = [Path(f).name[:-4] for f in trainedOn]
  if weightedOnDir != '':
    weightedOn = glob.glob(weightedOnDir+'/images/*.jpg')
    orpNames = orpNames + [Path(f).name[:-4] for f in weightedOn]

  mixedRand = []
  for ele in fullTrainSet:
    for ele2 in orpNames:
      if ele2 in ele:
        mixedRand.append(ele)
        continue
  randomPathes2 = [ele for ele in fullTrainSet if ele not in mixedRand]

  reducedRandomPathes2 = []
  for rp2 in randomPathes2:
    if os.path.exists(complete_data_folder+'labels/'+ Path(rp2).name[:-4] + '.txt'):
        reducedRandomPathes2.append(rp2)
  toGetUncertinityPath = reducedRandomPathes2
  print(len(toGetUncertinityPath))
  if fromRandSize < len(toGetUncertinityPath):
    toGetUncertinityPath = random.sample(reducedRandomPathes2, k=fromRandSize)
  print(len(toGetUncertinityPath))

  newFolderName = experimentName+'_Nominees'
  path_label = complete_iteration_folder+newFolderName+'/labels/'
  path_images = complete_iteration_folder+newFolderName+'/images/'
  os.makedirs(path_label) if not os.path.exists(path_label) else None
  os.makedirs(path_images) if not os.path.exists(path_images) else None

  for pth in toGetUncertinityPath:
    namef = Path(pth).name[:-4]
    if forWeights:
      src_path = complete_data_folder+'/labels/'+ namef+'.txt'
      dst_path = path_label + namef+'.txt'
      shutil.copy(src_path, dst_path)
    src_path = complete_data_folder+'/images/'+ namef+'.jpg'
    dst_path = path_images + namef+'.jpg'
    shutil.copy(src_path, dst_path)
  return path_images


#Test on Random subset
In this part we select some random validation subset to evaluate the peformance on and subsequently weigh the uncertinitiies of the different classes

In [ ]:
path_images_nominees_test = nominateImagesToObtainUncer(main_dir+main_exp_dir+previous_iteration_folder+previousExpfName,wexpName,fromRandSize=int(budgetPerIteration*0.1),forWeights=True)

In [ ]:
!unzip {main_dir+main_exp_dir+previous_iteration_folder}{previous_model_zip} -d /content/yolov7-AL/{previous_iteration_folder}

In [ ]:
#This somtims has to be updated manually based on the unzipped file above
weights_path = previous_iteration_folder+'/content/runs/detect/train/weights/best.pt'

In [ ]:
import yaml
config = {
         'path': '../datasets/VisDrone',
         'train': '/content/drive/MyDrive/AL/VOC2012/ValF/images',
         'val': '/content/drive/MyDrive/AL/VOC2012/DUA-WRP500/Iteration_6/w_iteration6DUA_5000_Nominees',
         'nc': 21,
         'names': [
    'none',
    'aeroplane',
    'bicycle',
    'bird',
    'boat',
    'bottle',
    'bus',
    'car',
    'cat',
    'chair',
    'cow',
    'diningtable',
    'dog',
    'horse',
    'motorbike',
    'person',
    'pottedplant',
    'sheep',
    'sofa',
    'train',
    'tvmonitor'
]}

with open("data.yaml", "w") as file:
    yaml.dump(config, file, default_flow_style=False)

In [ ]:
!python  test.py --save-txt  --weights {weights_path}   --img 416 --data data.yaml

#Calculating the uncertinity

In [ ]:
path_images_nominees = nominateImagesToObtainUncer(main_dir+main_exp_dir+previous_iteration_folder+previousExpfName,expName, fromRandSize = number_of_random_images_from_training)

In [ ]:
!python  detect.py --save-txt --save-conf --weights {weights_path}   --conf 0.05  --img 416 --source {main_dir+main_exp_dir+iteration_folder+expName+'_Nominees/images/'} --nosave


#Weighing the uncertinities

In [ ]:
expfName = 'iteration'+str(iteration)+approach+weighting_approach+'_'+str(number_of_random_images_from_training)
predFile = '/content/yolov7-AL/runs/detect/exp'
wafile = '../runs/detect/val/results.txt'


In [ ]:
previousFileList = glob.glob(main_dir+main_exp_dir+previous_iteration_folder+previousExpfName+'labels/*.txt')
trainOnDUA = getUncertinitiesForDir(predFile, valAccuracyDirF =wafile, method='DUA',budget = int(500),weighting = 3,)

#Construct the new train set for the next iteration

In [ ]:
newTraining(expfName,trainOnDUA,previousFileList)

In [ ]:
path_label = complete_iteration_folder+expfName+'/labels/'
path_images = complete_iteration_folder+expfName+'/images/'

wimg = glob.glob(complete_iteration_folder+wexpName+'_Nominees/images/*')
wtxt = glob.glob(complete_iteration_folder+wexpName+'_Nominees/labels/*')
for pth in wtxt:
  namef = Path(pth).name[:-4]
  src_path = complete_iteration_folder+wexpName+'_Nominees/labels/'+ namef+'.txt'
  dst_path = path_label + namef+'.txt'
  shutil.copy(src_path, dst_path)
for pth in wimg:
  namef = Path(pth).name[:-4]
  src_path = complete_iteration_folder+wexpName+'_Nominees/images/'+ namef+'.jpg'
  dst_path = path_images + namef+'.jpg'
  shutil.copy(src_path, dst_path)

In [ ]:
from google.colab import runtime
runtime.unassign()